In [2]:
import pandas as pd 
from gpt3forchem.api_wrappers import fine_tune, query_gpt3, extract_prediction
from gpt3forchem.data import get_waterstability_data
from sklearn.model_selection import train_test_split
from pycm import ConfusionMatrix
import time

import numpy as np

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = get_waterstability_data()

In [4]:
train_frame = pd.read_csv("/Users/kevinmaikjablonka/Downloads/rohitbatra1989-MOFWaterStability-67e1b6d/data/traindata.csv")
test_frame = pd.read_csv("/Users/kevinmaikjablonka/Downloads/rohitbatra1989-MOFWaterStability-67e1b6d/data/validationdata.csv")

In [5]:
train_frame

,ID,Common name,Stability,Activated formula unit_x,Metal,Nmetal,M_AtomicRadii,M_Electronegativity,M_Valency,M_Affinity,...,L_mfpMQNs40,L_mfpMQNs41,L_mfpMQNs42,L_mfpNumAliphaticRings,L_mfpNumAromaticRings,L_mfptpsa,L_linkermetalratio,L_no,L_noh,L_nh2o
0,1,Bio-MOF-14(95),3,Co2(AD)2(C4H9CO2)2,['Co'],['2'],1.35,1.88,2.0,0.66226,...,0.0,0.066667,0.033333,0.000000,0.066667,0.959495,2.000000,0.000000,0.000000,0.000000
1,2,"MIL-101(Cr)(28, 44, 49-51, 53, 62, 167)",3,Cr3F(H2O)2O(BDC)3,['Cr'],['3'],1.40,1.66,3.0,0.67500,...,0.0,0.000000,0.000000,0.000000,0.041667,0.777083,1.333333,0.333333,0.000000,0.666667
2,3,"MIL-101-SO3H (Cr)(51, 301)",3,Cr3F(H2O)2O(BDC-SO3H)3,['Cr'],['3'],1.40,1.66,3.0,0.67500,...,0.0,0.000000,0.000000,0.000000,0.034091,1.099176,1.333333,0.333333,0.000000,0.666667
3,4,"MIL-96(Al)(299, 302)",3,Al12O(OH)18(H2O)3(Al2(OH)4)(BTC)6,['Al'],['14'],1.25,1.61,3.0,0.43283,...,0.0,0.000000,0.000000,0.000000,0.047619,1.332143,0.428571,0.071429,1.571429,0.214286
4,5,N/A(261),3,Ni3(BTP)2,['Ni'],['3'],1.35,1.91,2.0,1.15716,...,0.0,0.000000,0.000000,0.000000,0.121212,0.651818,0.666667,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,203,Bio-MOF-11(95),0,Co2(AD)2(CH3CO2),['Co'],['2'],1.35,1.88,2.0,0.66226,...,0.0,0.088889,0.044444,0.000000,0.088889,1.282764,1.500000,0.000000,0.000000,0.000000
203,204,Bio-MOF-12(95),0,Co2(AD)2(C2H5CO2)2,['Co'],['2'],1.35,1.88,2.0,0.66226,...,0.0,0.066667,0.033333,0.029412,0.066667,0.796181,2.000000,0.000000,0.000000,0.000000
204,205,MIL-47(V)(268),0,V(O)BDC,['V'],['1'],1.35,1.63,5.0,0.52700,...,0.0,0.000000,0.000000,0.000000,0.055556,1.036111,1.000000,1.000000,0.000000,0.000000
205,206,MOF-505(63),0,Cu2(BPTC),['Cu'],['2'],1.35,1.90,2.0,1.23578,...,0.0,0.000000,0.000000,0.000000,0.058824,1.097059,0.500000,0.000000,0.000000,0.000000


In [6]:
train_data, test_data = train_test_split(data, train_size=0.8, random_state=42, stratify=data['stability'])

In [7]:
train_data

,common name,activated formula unita,confidence,stability,normalized_names
117,NU-1000-SALI-1 (104),Zr6(μ-OH)8(OH)8(TBAPY)2-CF3CO2,low,high,"Zr6(μ-OH)8(OH)8(1,3,6,8-tetrakis(p-benzoate)py..."
128,UiO-66-(CO2H)2 (322),Zr6O6(BDC-(CO2H)2)12,low,high,"Zr6O6(1,4-benzenedicarboxylate-(CO2H)2)12"
9,"MIL-100(Cr) (38, 39, 63)","Cr3OX(BTC)2, X&nbsp;=&nbsp;OH,&nbsp;F",medium,high,"Cr3O(OH)(1,3,5-benzenetricarboxylate)2"
35,"MIL-101-NH2(Cr) (51, 52)",Cr3F(H2O)2O(BDC-NH2)3,high,high,"Cr3F(H2O)2O(1,4-benzenedicarboxylate-NH2)3"
118,NU-1000-SALI-3 (104),Zr6(μ3-OH)8(OH)8(TBAPY)2-CF3(CF2)2CO2,low,high,"Zr6(μ3-OH)8(OH)8(1,3,6,8-tetrakis(p-benzoate)p..."
...,...,...,...,...,...
65,Zn-MOF-508 (77),Zn2(BDC)2(BPY),high,high,"Zn2(1,4-benzenedicarboxylate)2(4,4′-bipyridine)"
125,Tb-FTZB-MOF (321),((CH3)2NH2)2Tb6(μ3-OH)8(FTZB)6(H2O)6,low,high,((CH3)2NH2)2Tb6(μ3-OH)8(3-fluoro-4′-(2H-tetraz...
17,N/A (260),"Zn(1,4-BDP)",low,high,"Zn(1,4-benzenedi(4-pyrazolyl))"
187,Zr6O6(ABDC)12 (102),Zr6O6(ABDC)12,low,low,"Zr6O6(4,4′-azobenzenedicarboxylate)12"


In [9]:
data['stability'].value_counts()

high    146
low      42
Name: stability, dtype: int64

In [49]:
PROMPT_TEMPLATE_water_stability= "How is the water stability of {}"###"
COMPLETION_TEMPLATE_water_stability = "{}@@@"


def generate_water_stability_prompts(
    data: pd.DataFrame
) -> pd.DataFrame:
    prompts = []
    completions = []
    for i, row in data.iterrows():

        prompt = PROMPT_TEMPLATE_water_stability.format(
            row['normalized_names']
        )

        stability = 0 if row['stability'] == 'low' else 1
        completion = COMPLETION_TEMPLATE_water_stability.format(stability)
        prompts.append(prompt)
        completions.append(completion)

    prompts = pd.DataFrame(
        {"prompt": prompts, "completion": completions,}
    )

    return prompts

In [50]:
train_prompts = generate_water_stability_prompts(train_data)
test_prompts = generate_water_stability_prompts(test_data)

In [51]:
train_size = len(train_prompts)

filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename = f"run_files/{filename_base}_train_mof_water_stability_{train_size}.jsonl"

train_prompts.to_json(train_filename, orient="records", lines=True)


valid_filename = f"run_files/{filename_base}_valid_mof_water_stability_{train_size}.jsonl"

test_prompts.to_json(valid_filename, orient="records", lines=True)

In [52]:
modelname = fine_tune(train_filename, valid_filename)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [53]:
completions = query_gpt3(modelname, test_prompts)

In [54]:
predictions = [int(extract_prediction(completions, i)) for i in range(len(completions['choices']))]

In [57]:
cm = ConfusionMatrix(actual_vector=test_data['stability'].apply(lambda x: 0 if x == 'low' else 1).values, predict_vector=predictions)

In [58]:
print(cm)

Predict  0        1        
Actual
0        4        4        

1        2        28       





Overall Statistics : 

95% CI                                                            (0.72617,0.95804)
ACC Macro                                                         0.84211
ARI                                                               0.36192
AUNP                                                              0.71667
AUNU                                                              0.71667
Bangdiwala B                                                      0.79365
Bennett S                                                         0.68421
CBA                                                               0.6875
CSI                                                               0.4875
Chi-Squared                                                       8.91944
Chi-Squared DF                                                    1
Conditional Entropy                                             

### Try to add confidence scores to the predictions

In [59]:
PROMPT_TEMPLATE_water_stability= "How is the water stability of {}"###"
COMPLETION_TEMPLATE_water_stability_conf = "{} ({} confidence)@@@"


def generate_water_stability_prompts_confidence(
    data: pd.DataFrame
) -> pd.DataFrame:
    prompts = []
    completions = []
    for i, row in data.iterrows():

        prompt = PROMPT_TEMPLATE_water_stability.format(
            row['normalized_names']
        )

        stability = 0 if row['stability'] == 'low' else 1
        confidence = row['confidence']
        completion = COMPLETION_TEMPLATE_water_stability_conf.format(stability, confidence)
        prompts.append(prompt)
        completions.append(completion)

    prompts = pd.DataFrame(
        {"prompt": prompts, "completion": completions,}
    )

    return prompts

In [60]:
train_prompts_conf = generate_water_stability_prompts_confidence(train_data)
test_prompts_conf = generate_water_stability_prompts_confidence(test_data)

In [10]:
train_prompts_conf

NameError: name 'train_prompts_conf' is not defined

In [63]:
train_size = len(train_prompts)

filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
train_filename_w_conf = f"run_files/{filename_base}_train_conf_mof_water_stability_{train_size}.jsonl"

train_prompts_conf.to_json(train_filename_w_conf, orient="records", lines=True)


valid_filename_w_conf = f"run_files/{filename_base}_valid_conf_mof_water_stability_{train_size}.jsonl"

test_prompts_conf.to_json(valid_filename_w_conf, orient="records", lines=True)

In [64]:
modelname_w_conf = fine_tune(train_filename_w_conf, valid_filename_w_conf)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [66]:
completions_w_conf = query_gpt3(modelname_w_conf, test_prompts_conf)

In [88]:
test_prompts.iloc[0]['prompt']

'How is the water stability of Zr6O6(1,4-benzenedicarboxylate-Cl2)12'

In [68]:
completions_w_conf = [i['text'] for i in completions_w_conf['choices']]

In [77]:
predictions_w_conf = []
confidence_w_conf = []

for compl in completions_w_conf:
    compl = compl.split('@@@')[0]
    pred, conf = compl.split(' (')
    predictions_w_conf.append(int(pred))
    confidence_w_conf.append(conf[:-1])

predictions_w_conf = np.array(predictions_w_conf)
confidence_w_conf = np.array(confidence_w_conf)

In [92]:
print(ConfusionMatrix(test_data['confidence'].values, [s.split()[0] for s in confidence_w_conf]))

Predict      high         low          medium       
Actual
high         6            4            0            

low          4            17           0            

medium       1            5            1            





Overall Statistics : 

95% CI                                                            (0.47821,0.78495)
ACC Macro                                                         0.75439
ARI                                                               0.13316
AUNP                                                              0.64601
AUNU                                                              0.64073
Bangdiwala B                                                      0.4917
Bennett S                                                         0.44737
CBA                                                               0.44739
CSI                                                               0.25056
Chi-Squared                                                       10.66227
C

In [83]:
confidence_mask = confidence_w_conf=='high confidence'

In [80]:
true_values = test_data['stability'].apply(lambda x: 0 if x == 'low' else 1).values

In [84]:
confusion_matrix = ConfusionMatrix(actual_vector=true_values[confidence_mask], predict_vector=predictions_w_conf[confidence_mask])

In [85]:
print(confusion_matrix)

Predict 0       1       
Actual
0       3       1       

1       0       7       





Overall Statistics : 

95% CI                                                            (0.7392,1.07898)
ACC Macro                                                         0.90909
ARI                                                               0.6372
AUNP                                                              0.875
AUNU                                                              0.875
Bangdiwala B                                                      0.85294
Bennett S                                                         0.81818
CBA                                                               0.8125
CSI                                                               0.8125
Chi-Squared                                                       7.21875
Chi-Squared DF                                                    1
Conditional Entropy                                               0.29501
Crame